In [2]:
import pandas as pd
import newspaper as nwp

f = open('raw_data_cyware-api-14-08-2019.csv', 'r', encoding="utf-8")
df = pd.read_csv(f, index_col=0)
f.close()

In [ ]:
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
import datetime
from tqdm import tqdm


def fetch_and_analyze(df_row):
    text = ""
    
    try:
        url = df_row['web_sp_link']
        article = nwp.Article(url)
        article.download()
        
        # Wait for article to be downloaded (max 2s)
        max_time=0.0
        while((article.download_state == 0) & (max_time < 0.5)):
            time.sleep(0.1)
            max_time += 0.1

        # Parse article and scrap the text content
        article.parse()
        text = article.text
    except nwp.ArticleException:
        print("Article failed to download.\nUrl:{}\nTime: {}".format(url, str(datetime.datetime.now())))
    finally:
        df_row['text_scraped']=text
        return df_row 

scraped_articles = []
count = len(df.values)
num_cores = multiprocessing.cpu_count()

with tqdm(total=count) as pbar:
    while(count != 0):
        inputs = df[count-num_cores:count].to_dict('records')

        rows = Parallel(n_jobs=num_cores)(delayed(fetch_and_analyze)(i) for i in inputs)
        scraped_articles.extend(rows)
        
        pbar.update(len(rows))
        count -= len(rows)

In [ ]:
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
import datetime
from tqdm import tqdm
from newspaper import news_pool

def fetch_and_analyze(df_row):
    text = ""
    
    try:
        url = df_row['web_sp_link']
        article = nwp.Article(url)
        article.download()
        
        # Wait for article to be downloaded (max 2s)
        max_time=0.0
        while((article.download_state == 0) & (max_time < 0.5)):
            time.sleep(0.1)
            max_time += 0.1

        # Parse article and scrap the text content
        article.parse()
        text = article.text
    except nwp.ArticleException:
        print("Article failed to download.\nUrl:{}\nTime: {}".format(url, str(datetime.datetime.now())))
    finally:
        df_row['text_scraped']=text
        return df_row 

articles = df[0:2].to_dict('records')
scraped_articles = []
papers = []

for article in articles:
    papers.append(nwp.build(article['url']))

news_pool.set(papers, threads_per_source=1)
news_pool.join()

for i in range(len(papers)):
    articles[i]['text_scraped'] = papers[i].articles[0].text

In [3]:
from newspaper import news_pool

url1 = df.iloc[1]['web_sp_link']

paper1 = nwp.build(url1)

papers = [paper1]

news_pool.set(papers, threads_per_source=2)
news_pool.join()

In [5]:
paper1.description

'Hotel Data Breach - Largest lodging franchisors Choice Hotels has recently disclosed that it has suffered a data breach that occurred due to a misconfigured MongoDB database. Click here to know more!'

In [6]:
url1

'https://cyware.com/news/misconfigured-mongodb-database-leaks-700000-choice-hotels-customer-records-8894a252'

In [7]:
paper1.